### 네이버 쇼핑 키워드 인사이트

#### 개인과제
- 키워드에 대한 검색량 데이터 수집
- 검색어에 대한 상품등록수 수집
- 수집한 데이터에 대한 인사이트 도출 : 예시) 경쟁률

#### 조별과제
- 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드와 상품 찾기
    - 예시외의 추가적인 페이지 데이터 수집하여 프로젝트 수행가능

#### 참고 서비스
- 신사임당 유투브 : https://youtu.be/Z0hL9khbpIw
- 응용 서비스 : https://whereispost.com/keyword, https://itemscout.io

#### 데이터 수집 프로세스
1. 네이버 검색광고 회원가입 및 로그인 : https://searchad.naver.com
2. 광고 시스템 > 도구 > API 사용관리
3. CUSTOMER_ID, API_KEY, SECRET_KEY 획득
4. 네이버 검색 광고 API 매뉴얼 사이트 참고 : http://naver.github.io/searchad-apidoc/#/guides
5. github 페이지 참고
    - https://github.com/naver/searchad-apidoc
    - https://github.com/naver/searchad-apidoc/blob/master/python-sample/examples/ad_management_sample.py
6. 키워드 입력하면 해당 키워드에 대한 데이터 수집

- hint
    - github 페이지의 코드에서 signaturehelper.py 파일을 이용하여 header를 만들어야 합니다.

In [28]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
warnings.filterwarnings('ignore')

#### 1. 개별과제 : 키워드에 대한 검색량 데이터 수집

#### 1) 모듈과 함수 정의, API_KEY, SECRET_KEY, CUSTOMER_ID 입력

In [25]:
def get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

BASE_URL = 'https://api.naver.com'
API_KEY = '01000000002b4f24d4188c04e92f491665722e804510867e1c628dade9bc6d5b0b88df1c46'
SECRET_KEY = 'AQAAAAArTyTUGIwE6S9JFmVyLoBFNBwGkvs7j7eokzntbfuiEA=='
CUSTOMER_ID = '2488369'

#### 2) 키워드 도구 API를 사용해 키워드 조회

In [32]:
uri = '/keywordstool'
method = 'GET'
r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(input('연관키워드를 조회할 키워드를 입력하세요\n')),
                 headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

#### 3) 조회한 키워드 데이터 전처리

In [33]:
r.json()['keywordList'][0]

{'relKeyword': '그래픽카드',
 'monthlyPcQcCnt': 52600,
 'monthlyMobileQcCnt': 90800,
 'monthlyAvePcClkCnt': 131.9,
 'monthlyAveMobileClkCnt': 956.0,
 'monthlyAvePcCtr': 0.27,
 'monthlyAveMobileCtr': 1.1,
 'plAvgDepth': 15,
 'compIdx': '높음'}

In [15]:
df=pd.DataFrame(r.json()['keywordList'])
df.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,그래픽카드,52600,90800,131.9,956.0,0.27,1.10,15,높음
1,중고PC,2790,4100,74.3,270.2,2.89,6.96,15,높음
2,조립식컴퓨터,9880,32200,307.8,2294.0,3.37,7.63,15,높음
3,GTX1060,12700,21500,15.8,154.3,0.14,0.75,15,높음
4,컴퓨터가격,3400,13600,68.4,638.4,2.18,5.00,15,높음


In [16]:
df.rename({'compIdx':'경쟁정도',
           'monthlyAveMobileClkCnt':'월평균클릭수_모바일',
           'monthlyAveMobileCtr':'월평균클릭률_모바일',
           'monthlyAvePcClkCnt':'월평균클릭수_PC',
           'monthlyAvePcCtr':'월평균클릭률_PC', 
           'monthlyMobileQcCnt':'월간검색수_모바일',
           'monthlyPcQcCnt': '월간검색수_PC',
           'plAvgDepth':'월평균노출광고수', 
           'relKeyword':'연관키워드'},axis=1,inplace=True)
df.head()

,연관키워드,월간검색수_PC,월간검색수_모바일,월평균클릭수_PC,월평균클릭수_모바일,월평균클릭률_PC,월평균클릭률_모바일,월평균노출광고수,경쟁정도
0,그래픽카드,52600,90800,131.9,956.0,0.27,1.10,15,높음
1,중고PC,2790,4100,74.3,270.2,2.89,6.96,15,높음
2,조립식컴퓨터,9880,32200,307.8,2294.0,3.37,7.63,15,높음
3,GTX1060,12700,21500,15.8,154.3,0.14,0.75,15,높음
4,컴퓨터가격,3400,13600,68.4,638.4,2.18,5.00,15,높음


In [17]:
df=df[['연관키워드', '월간검색수_PC', '월간검색수_모바일',
       '월평균클릭수_PC','월평균클릭수_모바일', 
       '월평균클릭률_PC','월평균클릭률_모바일',
        '월평균노출광고수','경쟁정도']]
df.head()

,연관키워드,월간검색수_PC,월간검색수_모바일,월평균클릭수_PC,월평균클릭수_모바일,월평균클릭률_PC,월평균클릭률_모바일,월평균노출광고수,경쟁정도
0,그래픽카드,52600,90800,131.9,956.0,0.27,1.10,15,높음
1,중고PC,2790,4100,74.3,270.2,2.89,6.96,15,높음
2,조립식컴퓨터,9880,32200,307.8,2294.0,3.37,7.63,15,높음
3,GTX1060,12700,21500,15.8,154.3,0.14,0.75,15,높음
4,컴퓨터가격,3400,13600,68.4,638.4,2.18,5.00,15,높음


#### 4) 키워드 검색 도구 함수화

In [21]:
def keyword_master(keyword):
    uri = '/keywordstool'
    method = 'GET'
    r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
        headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    df=pd.DataFrame(r.json()['keywordList'])
    df.rename({'compIdx':'경쟁정도',
           'monthlyAveMobileClkCnt':'월평균클릭수_모바일',
           'monthlyAveMobileCtr':'월평균클릭률_모바일',
           'monthlyAvePcClkCnt':'월평균클릭수_PC',
           'monthlyAvePcCtr':'월평균클릭률_PC', 
           'monthlyMobileQcCnt':'월간검색수_모바일',
           'monthlyPcQcCnt': '월간검색수_PC',
           'plAvgDepth':'월평균노출광고수', 
           'relKeyword':'연관키워드'},axis=1,inplace=True)
    df=df[['연관키워드', '월간검색수_PC', '월간검색수_모바일',
       '월평균클릭수_PC','월평균클릭수_모바일', 
       '월평균클릭률_PC','월평균클릭률_모바일',
        '월평균노출광고수','경쟁정도']]
    return df

In [26]:
keyword = "그래픽카드"
df = keyword_master(keyword)

In [27]:
df.head()

,연관키워드,월간검색수_PC,월간검색수_모바일,월평균클릭수_PC,월평균클릭수_모바일,월평균클릭률_PC,월평균클릭률_모바일,월평균노출광고수,경쟁정도
0,그래픽카드,52600,90800,131.9,956.0,0.27,1.10,15,높음
1,중고PC,2790,4100,74.3,270.2,2.89,6.96,15,높음
2,조립식컴퓨터,9880,32200,307.8,2294.0,3.37,7.63,15,높음
3,GTX1060,12700,21500,15.8,154.3,0.14,0.75,15,높음
4,컴퓨터가격,3400,13600,68.4,638.4,2.18,5.00,15,높음


#### 2. 개별과제 : 검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [160]:
keyword = '그래픽카드'

In [168]:
uri = '/keywordstool'
method = 'GET'
r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
    headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

In [169]:
r.json()['keywordList'][0]

{'relKeyword': '그래픽카드',
 'monthlyPcQcCnt': 52600,
 'monthlyMobileQcCnt': 90800,
 'monthlyAvePcClkCnt': 131.9,
 'monthlyAveMobileClkCnt': 956.0,
 'monthlyAvePcCtr': 0.27,
 'monthlyAveMobileCtr': 1.1,
 'plAvgDepth': 15,
 'compIdx': '높음'}

In [170]:
df=pd.DataFrame(r.json()['keywordList'])
df.head(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,그래픽카드,52600,90800,131.9,956.0,0.27,1.10,15,높음
1,중고PC,2790,4100,74.3,270.2,2.89,6.96,15,높음


In [171]:
df2 = df[['relKeyword','monthlyPcQcCnt','monthlyMobileQcCnt']]
df2.head(2)

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt
0,그래픽카드,52600,90800
1,중고PC,2790,4100


In [172]:
df2.loc[df['monthlyPcQcCnt'] == '< 10']
df2.loc[df2.monthlyPcQcCnt == '< 10','monthlyPcQcCnt'] = 10

In [173]:
df2.loc[df['monthlyMobileQcCnt'] == '< 10']
df2.loc[df2.monthlyMobileQcCnt == '< 10','monthlyMobileQcCnt'] = 10

In [174]:
df2 = df2.astype({'monthlyPcQcCnt': int, 'monthlyMobileQcCnt': int}, errors = 'ignore')
df2

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt
0,그래픽카드,52600,90800
1,중고PC,2790,4100
2,조립식컴퓨터,9880,32200
3,GTX1060,12700,21500
4,컴퓨터가격,3400,13600
...,...,...,...
598,진해노트북수리,10,30
599,일산맥수리,10,10
600,조립노트북,380,1010
601,올인원PC수리,10,10


In [175]:
df2.dtypes

relKeyword            object
monthlyPcQcCnt         int32
monthlyMobileQcCnt     int32
dtype: object

In [176]:
df2['total_search_count'] = df2['monthlyPcQcCnt'] + df2['monthlyMobileQcCnt']
df2.head()

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,total_search_count
0,그래픽카드,52600,90800,143400
1,중고PC,2790,4100,6890
2,조립식컴퓨터,9880,32200,42080
3,GTX1060,12700,21500,34200
4,컴퓨터가격,3400,13600,17000


In [144]:
keyword = '그래픽카드'
uri = '/keywordstool'
method = 'GET'
r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
    headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
r.json()['keywordList'][0]['relKeyword']

'그래픽카드'

In [145]:
url = "https://search.shopping.naver.com/api/search/all?sort=rel&pagingIndex=1&pagingSize=40&viewType=list&productSet=total&deliveryFee=&deliveryTypeValue=&frm=NVSHTTL&query={keyword}&origQuery={keyword}&iq=&eq=&xq=&window="
response = requests.get(url)
response

<Response [200]>

In [150]:
data = response.json()["subFilters"][0]['filterValues'][0]['productCount']

In [147]:
data

580

In [155]:
def relKeyword(keyword):
    uri = '/keywordstool'
    method = 'GET'
    r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
        headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    df3 = pd.DataFrame(r.json()['keywordList'])
    df3 = df3['relKeyword']
    return df3
keyword = "그래픽카드"
df3 = relKeyword(keyword)
df3

0        그래픽카드
1         중고PC
2       조립식컴퓨터
3      GTX1060
4        컴퓨터가격
        ...   
598    진해노트북수리
599      일산맥수리
600      조립노트북
601    올인원PC수리
602    제주USB복구
Name: relKeyword, Length: 603, dtype: object

In [ ]:
rellist = df3.to_list()
rellist

In [158]:
item = []
def item_count(keyword='그래픽카드'):
    uri = '/keywordstool'
    method = 'GET'
    r = requests.get(BASE_URL + uri+'?hintKeywords={}&showDetail=1'.format(keyword),
        headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    r.json()['keywordList'][0]['relKeyword']
    
    for i in rellist:
        url = "https://search.shopping.naver.com/api/search/all?sort=rel&pagingIndex=1&pagingSize=40&viewType=list&productSet=total&deliveryFee=&deliveryTypeValue=&frm=NVSHTTL&query={i}&origQuery={i}&iq=&eq=&xq=&window="
        response = requests.get(url)
        data = response.json()["subFilters"][0]['filterValues'][0]['productCount']
        item.append(data)
    return item

In [ ]:
keyword = '그래픽카드'
item_count(keyword)

In [ ]:
df4 = pd.DataFrame(item, columns = ['item_count'])


In [ ]:
result_df = pd.concat([df2, df4])

In [22]:
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count
0,실내자전거,18300,159100,245638,177400
1,온라인PT,580,1600,11983,2180
2,하체운동기구,1070,7800,56684,8870
3,스피닝자전거,2380,25000,60342,27380
4,헬스자전거,1270,11500,243545,12770
5,홈트,4500,19900,924110,24400
6,스쿼트머신,2230,17100,45269,19330
7,다리운동기구,360,4110,73169,4470
8,실내운동기구,1530,13300,341120,14830
9,실내사이클,3600,32500,140261,36100


#### 3. 개별과제 : 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드
- 수집된 데이터에서 추가로 얻을수 있는 인사이트 추가
    - 논리적인 인사이트이면 인사이트 수준에 대한 평가는 하지 않음

In [177]:
# 현재 코드 오류로 아이템수 출력이 되지 않으나 주어진 결과로만 해석했을 때 키워드에 지역 이름이 들어가거나 PC가 들어갈 경우 경쟁률이 낮아짐을 보이고 있다
df2

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,total_search_count
0,그래픽카드,52600,90800,143400
1,중고PC,2790,4100,6890
2,조립식컴퓨터,9880,32200,42080
3,GTX1060,12700,21500,34200
4,컴퓨터가격,3400,13600,17000
...,...,...,...,...
598,진해노트북수리,10,30,40
599,일산맥수리,10,10,20
600,조립노트북,380,1010,1390
601,올인원PC수리,10,10,20


#### 4. 조별과제 : 스마트 스토어를 운영한다고 생각하고 경쟁력이 낮으면서 많은 수익을 낼수 있는 키워드 찾기
- 정답이 있는것은 아닙니다
- 여러가지 가정과 데이터를 수집, 분석하여 판매할 상품과 키워드를 선정해 보시면 됩니다.
- 앞서 2일간 했던 프로젝트와 같은 방법으로 수행하시면 됩니다.

### 평가기준

#### 개인평가(70)

    - 기본 문제 해결(30)
        - 데이터 수집 방법 : 10점
            - 최적의 수집 방법을 사용했는지 확인
        - 수집된 데이터의 결과 : 20점
            - 1번: 키워드에 대한 검색량 데이터 수집 : 15점
            - 2번: 검색어에 대한 상품등록수 수집 : 5점
            
    - 적절한 주석 및 설명(10)
        - 코드의 가독성 및 코드의 설명 : 10점
        
    - 코드 오류(10)
        - 발생 오류마다 : -2점
        
    - 추가 분석(20)
        - 인사이트 도출
            - 경쟁률 데이터 출력 : 10점
            - 추가 인사이트 : 10점
            
#### 조별평가(30)
    
    - 비지니스 인사이트 도출(5)
        - 스마트 스토어 운영자 입장에서의 추가 인사이트 도출
        - 예시) 선정한 상품으로 소호몰에서 1위 하였을경우 예상되는 매출
        - 인사이트 도출을 위한 프로세스
        
    - 실습문제의 분석(5)
        - 제출 자료가 잘 정리되고 판매 상품과 키워드를 논리적으로 잘 선정
        - 상품과 키워드 선정 자동화 > 가독성 좋은 키워드와 상품 선정 프로세스
        
    - 실습문제의 적절한 정리(20)
        - 프로젝트 주제(상품과 키워드 선정)에 맞는 논리적 인사이트가 있는지 확인
        - 프로젝트 수행 후 추후 과제 및 느낀점